In [1]:
import numpy as np
import sys
sys.path.append('../../')
sys.path.append('../../pyfos/')
from pyfos.models.feedforward import build_feedforward
from pyfos.models.var_feedforward import build_var_feedforward
from pyfos.generic.optimizer import gd_optimizer
#from pyfos.optimizers.greed_optimize import greed_optimize
from pyfos.generic.regularizers import gaus_prior
from functools import partial 
from pyfos.tc.simple import  simple_tc
import theano
import theano.tensor as T
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
X_train, X_test = np.load('../../../data/mnist_train_x.npy'), np.load('../../../data/mnist_test_x.npy')
Y_train, Y_test = np.load('../../../data/mnist_train_y.npy'), np.load('../../../data/mnist_test_y.npy')

X = T.matrix()
#X_train = X_train**2
#X_test = X_test**2

In [ ]:
X_train.shape

In [7]:
batch_size = 100
lr = theano.shared(10**(-6))
log_alphas = theano.shared(np.array([100.0]))
inits = [np.sqrt(2.0/784), np.sqrt(2.0/300)]
optimizer = partial(gd_optimizer, learning_rate=lr)
model_build = partial(build_feedforward, init_sigmas=inits, structure = [X_train.shape[1], 300, 10],
                      nonlinearity=lambda x:T.nnet.relu(x), use_softmax=True, 
                      log_alphas=log_alphas, bias=True)
training_procedure = simple_tc( model_build, optimizer, X_train, Y_train,  batch_size=batch_size )
model = model_build(params=training_procedure.models[0].params, dataset_size=X_train.shape[0])
X = T.matrix()
Y = T.ivector()

predict = theano.function([X], model.predict_var(X))
wtf = theano.function([X, Y], model.validation(X,Y))

In [5]:
Y = predict(X_test)

In [13]:
def logsumexp(X, axis):
        max_X = T.max(X)
        return max_X + T.log(T.sum(T.exp(X - max_X), axis=axis, keepdims=True))
sm =  (Y - logsumexp(Y, 1)).eval()
Y_test = Y_test.astype(np.uint)

print T.sum(sm[T.arange(Y_test.shape[0]).eval(), Y_test.tolist()]).eval() 

ValueError: Not enough dimensions on Elemwise{exp,no_inplace}.0 to reduce on axis 1

In [ ]:
wtf(X_test, Y_test)

In [ ]:
#100 epochs, 2500/25 ~ 100 iterations
err = []
Y = predict(X_test)
err.append(np.mean(np.not_equal(Y, Y_test)))
print err[-1]
print wtf(X_test,Y_test)
for epoch in xrange(25):
    
    for i in xrange(500):
        res = training_procedure.do_train()
        
    Y = predict(X_test)
    err.append(np.mean(np.not_equal(Y, Y_test)))
    print wtf(X_test,Y_test)
    print 'acc', err[-1], training_procedure.do_validation()

0.9575
-146230.844799
-17432.936246
acc 0.0821 -18112.4731022


In [ ]:
err

In [ ]:
plt.plot(err)
print err[-1], np.min(err)

In [ ]:
for i in [-2 , 15, 271, -1, -5, 123]:
    plt.plot(X_test[i])
    plt.scatter(200, Y_test[i])
    plt.scatter(202, predict([X_test[i]]), c='r')
    plt.show()

In [6]:
batch_size = 25
hid = 10
lr = theano.shared(10**(-6))
log_alphas = theano.shared(np.array([0]))
inits = np.sqrt([2.0/X_test.shape[1], 2.0/hid])

optimizer = partial(gd_optimizer, learning_rate=lr)
model_build = partial(build_feedforward,  structure = [X_train.shape[1], hid,   1],  init_sigmas=inits, nonlinearity=lambda x:T.nnet.relu(x), log_alphas=log_alphas, bias=True)
training_procedure = simple_tc( model_build, optimizer, X_train, Y_train,   batch_size=batch_size )
model = model_build(params=training_procedure.models[0].params, dataset_size=1)
X = T.matrix()
predict = theano.function([X], model.predict_var(X))

60000
60000


In [21]:
#100 epochs, 2500/25 ~ 100 iterations
err = []
Y = predict(X_test)
err.append(mse(Y[:,0], Y_test))
print err[-1]
for epoch in xrange(25):
    
    for i in xrange(100):
        res =    training_procedure.do_train()
    #print res
    Y = predict(X_test)
    err.append(mse(Y[:,0], Y_test))
    #if epoch%10==0:
    
    print err[-1], res


NameError: name 'mse' is not defined

In [ ]:
batch_size = 25
hid = 10
lr = theano.shared(10**(-6))
param_num = X_train.shape[1]*hid+hid+hid*1+1
log_alphas = theano.shared(np.array([6]*param_num))
inits = np.sqrt([2.0/X_test.shape[1], 2.0/hid])

optimizer = partial(gd_optimizer, learning_rate=lr)
model_build = partial(build_var_feedforward, param_pool_size=25,
                      structure = [X_train.shape[1], hid,   1],  init_sigmas=inits, nonlinearity=lambda x:T.nnet.relu(x), 
                      log_alphas=log_alphas, bias=True, log_var_sigmas=np.array([-2.0]*param_num))
training_procedure = simple_tc( model_build, optimizer, X_train, Y_train,   batch_size=batch_size )
model = model_build(params=training_procedure.models[0].params, dataset_size=1)
X = T.matrix()
predict = theano.function([X], model.predict_var(X))

In [ ]:
#100 epochs, 2500/25 ~ 100 iterations
err = []
Y = predict(X_test)
err.append(mse(Y[:,0], Y_test))
print err[-1]
for epoch in xrange(50):
    
    for i in xrange(100):
        res =    training_procedure.do_train()
    #print res
    Y = predict(X_train)
    err.append(mse(Y[:,0], Y_train))
    #if epoch%10==0:
    print err[-1], res, np.sum((Y[:, 0]-Y_train)**2)


In [ ]:
np.min(model.params.eval()[param_num:])